# Chatbot reviewer with Llama3 70B and RAG
## Intro: The main goal of this research is to create an AI reviewer of one company which accumulates all the reviews of this company in 2023 and can answer the questions like: "what are the best parts of the service?", "what are the worst?" and solve entrepreneurs' problem of their business understanding.
#### Credits: 
#### 1) This research is inspired by DataFest 2024 Artem Sebalo's report named "Advanced application of GigaChat in Rabota.ru. Synthetic, search".
#### 2) Original dataset from https://github.com/yandex/geo-reviews-dataset-2023

## Data handling

### Some EDA

In [4]:
import pandas as pd

In [5]:
# read tskv file
df = pd.read_csv('geo-reviews-dataset-2023.tskv', sep='\t')
df.head()

,"address=Екатеринбург, ул. Московская / ул. Волгоградская / ул. Печатников",name_ru=Московский квартал,rating=3.,rubrics=Жилой комплекс,"text=Московский квартал 2.\nШумно : летом по ночам дикие гонки. Грязно : кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. Детские площадки убогие, на большой площади однотипные конструкции. Очень дорогая коммуналка. Часто срабатывает пожарная сигнализация. Жильцы уже не реагируют. В это время, обычно около часа, не работают лифты. Из плюсов - отличная планировка квартир ( Московская 194 ), на мой взгляд. Ремонт от застройщика на 3-. Окна вообще жуть - вместо вентиляции. По соотношению цена/качество - 3."
0,"address=Московская область, Электросталь, прос...",name_ru=Продукты Ермолино,rating=5.,rubrics=Магазин продуктов;Продукты глубокой за...,"text=Замечательная сеть магазинов в общем, хор..."
1,"address=Краснодар, Прикубанский внутригородско...",name_ru=LimeFit,rating=1.,rubrics=Фитнес-клуб,"text=Не знаю смутят ли кого-то данные правила,..."
2,"address=Санкт-Петербург, проспект Энгельса, 11...",name_ru=Snow-Express,rating=4.,rubrics=Пункт проката;Прокат велосипедов;Сапсё...,text=Хорошие условия аренды. \nДружелюбный пер...
3,"address=Тверь, Волоколамский проспект, 39",name_ru=Студия Beauty Brow,rating=5.,"rubrics=Салон красоты;Визажисты, стилисты;Сало...",text=Топ мастер Ангелина топ во всех смыслах )...
4,"address=Иркутская область, Черемхово, Первомай...",name_ru=Tele2,rating=5.,rubrics=Оператор сотовой связи;Интернет-провайдер,"text=Приятное общение, все доступно объяснили,..."


In [6]:
# wow, not comfortable names for cols, change it right now
df.columns = ['address', 'name_ru', 'rating', 'rubrics', 'text']

In [7]:
df.head()

,address,name_ru,rating,rubrics,text
0,"address=Московская область, Электросталь, прос...",name_ru=Продукты Ермолино,rating=5.,rubrics=Магазин продуктов;Продукты глубокой за...,"text=Замечательная сеть магазинов в общем, хор..."
1,"address=Краснодар, Прикубанский внутригородско...",name_ru=LimeFit,rating=1.,rubrics=Фитнес-клуб,"text=Не знаю смутят ли кого-то данные правила,..."
2,"address=Санкт-Петербург, проспект Энгельса, 11...",name_ru=Snow-Express,rating=4.,rubrics=Пункт проката;Прокат велосипедов;Сапсё...,text=Хорошие условия аренды. \nДружелюбный пер...
3,"address=Тверь, Волоколамский проспект, 39",name_ru=Студия Beauty Brow,rating=5.,"rubrics=Салон красоты;Визажисты, стилисты;Сало...",text=Топ мастер Ангелина топ во всех смыслах )...
4,"address=Иркутская область, Черемхово, Первомай...",name_ru=Tele2,rating=5.,rubrics=Оператор сотовой связи;Интернет-провайдер,"text=Приятное общение, все доступно объяснили,..."


In [8]:
# we're going to choose only one company to test our model and check hypothesis of the report of creating such an
# essence which can be a reviwer of your company
# let us count some values
df['name_ru'].value_counts()

name_ru
name_ru=Пятёрочка                                                6030
name_ru=Магнит                                                   2611
name_ru=Красное&Белое                                            1732
name_ru=Wildberries                                              1698
name_ru=Ozon                                                     1494
                                                                 ... 
name_ru=Лоза-Кизлярского Коньячного Завода                          1
name_ru=Car-Gold                                                    1
name_ru=Фрейми                                                      1
name_ru=Рыбачим вместе                                              1
name_ru=Дворец культуры авиастроителей имени 50-летия Октября       1
Name: count, Length: 148460, dtype: int64

In [9]:
# we don't have much compute so we only can choose not popular companies with not so many reviews.
df['name_ru'].value_counts()[150:160]

name_ru
name_ru=Fitness House              153
name_ru=Хачапури тетушки Марико    152
name_ru=Парк Краснодар             151
name_ru=Папа Джонс                 151
name_ru=Чио Чио                    150
name_ru=Апельсин                   150
name_ru=Волна                      150
name_ru=Мореон                     149
name_ru=Русь                       149
name_ru=BRITVA                     148
Name: count, dtype: int64

In [10]:
# ok, let it be park Krasnodar with 151 reviews.
df = df[df['name_ru'] == 'name_ru=Парк Краснодар']

In [11]:
df.rating.value_counts()

rating
rating=5.    142
rating=4.      8
rating=3.      1
Name: count, dtype: int64

In [12]:
# wow this park has a good rating, we won't see yelling and blaming here. It's for better! We'll see many benefits and 
# good mood:)
df.head()

,address,name_ru,rating,rubrics,text
152007,"address=Краснодар, Городской сад",name_ru=Парк Краснодар,rating=5.,rubrics=Парк культуры и отдыха,"text=Отличный парк, для всей семьи. Гулять мож..."
152008,"address=Краснодар, Городской сад",name_ru=Парк Краснодар,rating=5.,rubrics=Парк культуры и отдыха,"text=Парк Краснодара, очень красивое и уютное ..."
152009,"address=Краснодар, Городской сад",name_ru=Парк Краснодар,rating=5.,rubrics=Парк культуры и отдыха,"text=Самый лучший парк, который я посещала. Ог..."
152010,"address=Краснодар, Городской сад",name_ru=Парк Краснодар,rating=5.,rubrics=Парк культуры и отдыха,text=Удивительная красота!! Мир цветов и зелен...
152011,"address=Краснодар, Городской сад",name_ru=Парк Краснодар,rating=5.,rubrics=Парк культуры и отдыха,text=Не иначе как это чудо для нашей страны. Т...


In [13]:
# delete each column name inside of each row
def clean_row(df, col):
    df[col] = df[col].apply(lambda x: x.split('=')[1])
    return df

In [14]:
for col in df.columns:
    clean_row(df, col)

In [15]:
df

,address,name_ru,rating,rubrics,text
152007,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,"Отличный парк, для всей семьи. Гулять можно це..."
152008,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,"Парк Краснодара, очень красивое и уютное место..."
152009,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,"Самый лучший парк, который я посещала. Огромна..."
152010,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,Удивительная красота!! Мир цветов и зелени! По...
152011,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,Не иначе как это чудо для нашей страны. Такого...
...,...,...,...,...,...
152153,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,Вкусное мороженое возле японского сада)
152154,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,"Самый блестящий, ухоженный, просто замечательн..."
152155,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,Потрясающий парк. Гуляли весь день. Сына с тру...
152156,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,Шикарное место для прогулок на свежем воздухе....


In [16]:
# great, we've cleaned our df a bit. Describe df
df.describe(include='all')

,address,name_ru,rating,rubrics,text
count,151,151,151,151,151
unique,1,1,3,1,151
top,"Краснодар, Городской сад",Парк Краснодар,5.,Парк культуры и отдыха,"Отличный парк, для всей семьи. Гулять можно це..."
freq,151,151,142,151,1


In [17]:
# we can delete unnecessary columns since we have unique values in them
df_clean = df.drop(columns=['address', 'name_ru', 'rubrics'])

In [18]:
df_clean

,rating,text
152007,5.,"Отличный парк, для всей семьи. Гулять можно це..."
152008,5.,"Парк Краснодара, очень красивое и уютное место..."
152009,5.,"Самый лучший парк, который я посещала. Огромна..."
152010,5.,Удивительная красота!! Мир цветов и зелени! По...
152011,5.,Не иначе как это чудо для нашей страны. Такого...
...,...,...
152153,5.,Вкусное мороженое возле японского сада)
152154,5.,"Самый блестящий, ухоженный, просто замечательн..."
152155,5.,Потрясающий парк. Гуляли весь день. Сына с тру...
152156,5.,Шикарное место для прогулок на свежем воздухе....


In [19]:
# what if we combine two remaining columns so our each review would have its rating and a review itself?
# i think it's a great idea!
df_clean['review'] = df_clean.apply(lambda x: 'Оценка: ' + x['rating'] + ' ' + x['text'], axis=1)

In [20]:
df_clean

,rating,text,review
152007,5.,"Отличный парк, для всей семьи. Гулять можно це...","Оценка: 5. Отличный парк, для всей семьи. Гуля..."
152008,5.,"Парк Краснодара, очень красивое и уютное место...","Оценка: 5. Парк Краснодара, очень красивое и у..."
152009,5.,"Самый лучший парк, который я посещала. Огромна...","Оценка: 5. Самый лучший парк, который я посеща..."
152010,5.,Удивительная красота!! Мир цветов и зелени! По...,Оценка: 5. Удивительная красота!! Мир цветов и...
152011,5.,Не иначе как это чудо для нашей страны. Такого...,Оценка: 5. Не иначе как это чудо для нашей стр...
...,...,...,...
152153,5.,Вкусное мороженое возле японского сада),Оценка: 5. Вкусное мороженое возле японского с...
152154,5.,"Самый блестящий, ухоженный, просто замечательн...","Оценка: 5. Самый блестящий, ухоженный, просто ..."
152155,5.,Потрясающий парк. Гуляли весь день. Сына с тру...,Оценка: 5. Потрясающий парк. Гуляли весь день....
152156,5.,Шикарное место для прогулок на свежем воздухе....,Оценка: 5. Шикарное место для прогулок на свеж...


In [21]:
# yep, we did it. Delete text and rating cols
df_clean = df_clean.drop(columns=['rating', 'text'])

In [22]:
df_clean = df_clean.reset_index(drop=True)

In [23]:
df_clean

,review
0,"Оценка: 5. Отличный парк, для всей семьи. Гуля..."
1,"Оценка: 5. Парк Краснодара, очень красивое и у..."
2,"Оценка: 5. Самый лучший парк, который я посеща..."
3,Оценка: 5. Удивительная красота!! Мир цветов и...
4,Оценка: 5. Не иначе как это чудо для нашей стр...
...,...
146,Оценка: 5. Вкусное мороженое возле японского с...
147,"Оценка: 5. Самый блестящий, ухоженный, просто ..."
148,Оценка: 5. Потрясающий парк. Гуляли весь день....
149,Оценка: 5. Шикарное место для прогулок на свеж...


In [24]:
# save our reviews as a .csv file
df_clean.to_csv('reviews.csv', index=False, sep=',')

## RAG implementation

In [25]:
# ideally use llama3 embeddings instead of phi3
import os
from dotenv import load_dotenv
from operator import itemgetter
from langchain import PromptTemplate
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain.load import dumps, loads
from langchain_pinecone import PineconeVectorStore

load_dotenv() # load env for api keys etc

# here we go
LLM = 'llama3-70b-8192' # the smartest model in groq api IMAO
embeddings_model = 'phi3:latest' # i've downloaded it previously for fun, so why not using it right now for embeddings purposes

# init model from groq
chat = ChatGroq(temperature=0.3, model_name=LLM) # we want mid temp

# create custom instructions
system = 'Ты мастер делать саммари и удобные выжимки из текста, отвечай только то, что ты видишь в контексте, отвечай всегда только на русском языке.'
human = '{text}'
prompt = ChatPromptTemplate.from_messages([('system', system), ('human', human)]) 

# init embeds
embeddings = OllamaEmbeddings(model=embeddings_model)

# create prompt-chat chain
model = prompt | chat
model.invoke({'text': 'пошути о чем-нибудь'})

/Users/imac/llm/.venv/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


AIMessage(content='Хорошо! Вот моя шутка:\n\nКто придумал понятие "ранний подъем"? Это же очевидно, что ночь была создана для сна, а не для подъема!', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 66, 'total_tokens': 110, 'completion_time': 0.125083898, 'prompt_time': 0.015309251, 'queue_time': None, 'total_time': 0.140393149}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_abd29e8833', 'finish_reason': 'stop', 'logprobs': None}, id='run-cb33850a-36a1-4203-a45f-50c620ef9358-0')

In [26]:
# previously we had AIMessage with all metadata, now we want to have only answer w/o any additional info. We're gonna use Parser from langchain for these purposes

parser = StrOutputParser()

chain = model | parser
chain.invoke({'text': 'Расскажи шутку'})

'Почему компьютер пошел к врачу? Он чувствовал себя немного глюком!'

In [27]:
# create template for model to see context
template = '''
Отвечай на вопросы на основе контекста ниже. Если не можешь ответить на вопрос, отвечай "Я не знаю"
Контекст: {context}
Вопрос: {question}
'''

prompt = PromptTemplate.from_template(template)
prompt.format(context='Такой контекст', question='Такой вопрос')

'\nОтвечай на вопросы на основе контекста ниже. Если не можешь ответить на вопрос, отвечай "Я не знаю"\nКонтекст: Такой контекст\nВопрос: Такой вопрос\n'

In [28]:
# we have a template for context, then we throw it into a model which has custom prompt and uses chatgroq to answer, then this all to be parsed to remove metadata. Let us check what will be the answer with our context
chain = prompt | model | parser

chain.invoke({"context": "Компания ООО 'ФЫВЛАПР' была основана в 1990 году и с тех пор развивалась с огромной скоростью", "question": "Как называется компания'?"})

"Компания называется ООО 'ФЫВЛАПР'."

In [29]:
# take a loader from langchain community and load our csv file
loader = CSVLoader('reviews.csv')
reviews_data = loader.load()

In [30]:
# have a glance at first 10 docs
reviews_data[:10]

[Document(page_content='review: Оценка: 5. Отличный парк, для всей семьи. Гулять можно целый день и вряд ли надоест. Чисто, везде охрана, есть кофетерии, много лавочек, да и вообще мест для сидения и даже лежания 😁 Удивительно, что городские власти до сих пор не прибрали его к рукам и не сделали платным. Жаль что не все наши власть имущие такие же как Сергей Николаевич. Тем, кто до сих пор раздумывает, ехать в парк или нет? – однозначно ЕЗЖАЙТЕ, не пожалеете.', metadata={'source': 'reviews.csv', 'row': 0}),
 Document(page_content='review: Оценка: 5. Парк Краснодара, очень красивое и уютное место, где можно от души поваляться на травке, и никто ничего не скажет, а главное вещи все чистые остаются, много разных арт объектов, и композиций, и стадион очень гармонично смотрится в парке. Рекомендую очень понравилось и взрослым и детям', metadata={'source': 'reviews.csv', 'row': 1}),
 Document(page_content='review: Оценка: 5. Самый лучший парк, который я посещала. Огромная территория, и в ско

In [31]:
# we are going to define vectorstore to convert our reviews into phi3 embeddings. After that we will retrieve relevant embeddings from this vectorstore and put into chain. Firstly we are using some part of them, 
# just to have a shot
vectorstore = DocArrayInMemorySearch.from_documents(reviews_data[:20], embedding=embeddings)

In [32]:
retriever = vectorstore.as_retriever() # define vectorstore as a retriever
retriever.invoke('что говорится о размерах парка?')

[Document(page_content='review: Оценка: 5. Очень красивый парк, огромный, многоуровневый, шикарные магнолии и золотые карпы , есть где отдохнуть , посидеть в тени деревьев, и прикоснуться к воде.', metadata={'source': 'reviews.csv', 'row': 7}),
 Document(page_content='review: Оценка: 5. Отличный парк, для всей семьи. Гулять можно целый день и вряд ли надоест. Чисто, везде охрана, есть кофетерии, много лавочек, да и вообще мест для сидения и даже лежания 😁 Удивительно, что городские власти до сих пор не прибрали его к рукам и не сделали платным. Жаль что не все наши власть имущие такие же как Сергей Николаевич. Тем, кто до сих пор раздумывает, ехать в парк или нет? – однозначно ЕЗЖАЙТЕ, не пожалеете.', metadata={'source': 'reviews.csv', 'row': 0}),
 Document(page_content='review: Оценка: 5. Шикарный парк. Галицкий молодец. Хоть кто-то для города что-то делает. Единственное ну очень тяжко обойти весь парк пешком. Поставьте в аренду самокаты.', metadata={'source': 'reviews.csv', 'row': 9}

In [33]:
# create new chain considering vector store.
chain = (
    {
        'context': itemgetter('question') | retriever, # put context into retriever
        'question': itemgetter('question'),
    }
    | prompt
    | model 
    | parser
)

In [34]:
# Checking how it works
questions = [
    'есть ли в парке мороженое?',
    'можно ли покататься на самокате?',
    'Есть ли минусы у этого парка?',
    'что понравилось больше всего, дай топ-5 причин?',
    'что не понравилось больше всего, дай топ-5 причин?',
    'Галицкий, кто это?'
]

for question in questions:
    print(f"Вопрос: {question}")
    print(f"Ответ: {chain.invoke({'question': question})}")

Вопрос: есть ли в парке мороженое?
Ответ: Я не знаю.
Вопрос: можно ли покататься на самокате?
Ответ: Я не знаю.
Вопрос: Есть ли минусы у этого парка?
Ответ: Да, есть минусы у этого парка. Некоторые посетители отметили, что в парке мало тени, и летом очень жарко, а также что тяжело обойти весь парк пешком.
Вопрос: что понравилось больше всего, дай топ-5 причин?
Ответ: Основываясь на контексте, можно выделить следующие топ-5 причин, которые понравились больше всего:

1. Приятная атмосфера и свежий воздух
2. Красивый японский сад
3. Вкусный кофе и какао
4. Чистота и безопасность (охрана, лавочки, места для сидения и лежания)
5. Красота и размер парка (огромный, многоуровневый, шикарные магнолии и золотые карпы)
Вопрос: что не понравилось больше всего, дай топ-5 причин?
Ответ: Я не знаю. В тексте нет упоминаний о чем-то, что не понравилось. Все отзывы положительные и содержат похвалу парку.
Вопрос: Галицкий, кто это?
Ответ: Галицкий - это человек, который что-то сделал для города, создал ш

In [36]:
# some questions seemed for LLM too specific, like an ice-cream or scooters existence. I know that some reviews have these words inside, but maybe temperature is too low, we will try to fix this later.
# but the best thing is that many answers seem very correct!
# we can proceed with batches as well
# define time.sleep to make some pause between huge amount of tokens' request from groq's api
import time

time.sleep(40)
chain.batch([{"question": q} for q in questions])

['Я не знаю.',
 'Я не знаю.',
 'Да, есть минусы у этого парка. Некоторые посетители отмечали, что парк очень большой и тяжело обойти его пешком, а также что летом в парке очень жарко и мало мест, где можно укрыться от солнца.',
 'Вот топ-5 причин, что понравилось больше всего, основываясь на контексте:\n\n1. Приятная атмосфера и свежий воздух\n2. Красивый японский сад\n3. Вкусный кофе и какао\n4. Огромный и шикарный парк с магнолиями и золотыми карпами\n5. Чистота и безопасность парка, с наличием кофетерий и мест для сидения и лежания',
 'Я не знаю.',
 'Галицкий - это человек, который сделал что-то полезное для города, создал шикарный парк.']

In [37]:
# there are controversial answers about minuses and top-5 problems. Disadvantages in the third q are ok but in 5th it says i don't know so it's ok but not the best answer
# streaming is cool feature, have a look
for s in chain.stream({"question": "детям понравится? Расскажи подробнее, что там можно делать детям?"}):
    print(s, end="", flush=True)

Дети, вероятно, будут рады в этом парке, потому что он предлагает различные возможности для отдыха и развлечений. Там есть кофетерии, лавочки и места для сидения, где можно отдохнуть. Кроме того, парк является большим и имеет различные уровни, что может быть интересно для детей. Однако, конкретных детских площадок или аттракционов в тексте не упоминается.

In [38]:
# wow just have a look on those park details. Great! Love it. Let's indicate vectorstore so we won't store in memory all the embeddings.
index_name = "reviews"

pinecone = PineconeVectorStore.from_documents(
    reviews_data, embeddings, index_name=index_name
) 

In [39]:
# let us check cosine similarity score for our embeddings
pinecone.similarity_search_with_score('карпы')

[(Document(page_content='review: Оценка: 5. Супер! Спасибо большое Сергею Галицкому. В любое время года стоит посетить. Восхищает место. Японский сад - изюминка на торте', metadata={'row': 119.0, 'source': 'reviews.csv'}),
  0.656286836),
 (Document(page_content='review: Оценка: 5. Прогулка всей семьи! Удобная парковка; великолепный парк, отлично сделан. Ухаживают , убирают, заботиться . Много детских площадок. Мало скамеек для отдыха и урн для мусора', metadata={'row': 109.0, 'source': 'reviews.csv'}),
  0.625989497),
 (Document(page_content='review: Оценка: 5. Обожаю этот парк за его удивительную красоту, чистоту и ухоженность. Прекрасные живописные виды. Редкие и красивые растения. Необычные водоёмы, фонтаны и волшебная подсветка. Очень подходит для прогулок. Можно с детьми. Есть детская площадка, кафе и туалеты', metadata={'row': 94.0, 'source': 'reviews.csv'}),
  0.610643506),
 (Document(page_content='review: Оценка: 5. Классный парк, очень красиво, продаётся прекрасное натурально

In [40]:
setup = RunnableParallel(context=retriever, question=RunnablePassthrough())
setup.invoke('Тень есть, чтобы скрыться?')

{'context': [Document(page_content='review: Оценка: 5. Отличный парк, для всей семьи. Гулять можно целый день и вряд ли надоест. Чисто, везде охрана, есть кофетерии, много лавочек, да и вообще мест для сидения и даже лежания 😁 Удивительно, что городские власти до сих пор не прибрали его к рукам и не сделали платным. Жаль что не все наши власть имущие такие же как Сергей Николаевич. Тем, кто до сих пор раздумывает, ехать в парк или нет? – однозначно ЕЗЖАЙТЕ, не пожалеете.', metadata={'source': 'reviews.csv', 'row': 0, 'text': 'review: Оценка: 5. Отличный парк, для всей семьи. Гулять можно целый день и вряд ли надоест. Чисто, везде охрана, есть кофетерии, много лавочек, да и вообще мест для сидения и даже лежания 😁 Удивительно, что городские власти до сих пор не прибрали его к рукам и не сделали платным. Жаль что не все наши власть имущие такие же как Сергей Николаевич. Тем, кто до сих пор раздумывает, ехать в парк или нет? – однозначно ЕЗЖАЙТЕ, не пожалеете.'}),
  Document(page_content=

In [41]:
# define new chain and add pinecone retriever as retriever and pass runnable
chain = (
    {'context': pinecone.as_retriever(), 'question': RunnablePassthrough()}
    | prompt
    | model
    | parser
)

chain.invoke('есть японский сад?')

'Да, есть японский парк.'

In [42]:
questions = [
    'есть ли в парке мороженое?',
    'можно ли покататься на самокате?',
    'Есть ли минусы у этого парка?',
    'что понравилось больше всего, дай топ-5 причин?',
    'что не понравилось больше всего, дай топ-5 причин?',
    'Галицкий, кто это?'
]

for question in questions:
    print(f'Вопрос: {question}')
    print(f"Ответ: {chain.invoke({'question': question})}")

Вопрос: есть ли в парке мороженое?
Ответ: Да, в парке есть мороженое, по крайней мере, возле японского сада.
Вопрос: можно ли покататься на самокате?
Ответ: Да, можно покататься на самокате.
Вопрос: Есть ли минусы у этого парка?
Ответ: Да, есть минусы у этого парка. Один из минусов - это отсутствие табличек с информацией о растениях.
Вопрос: что понравилось больше всего, дай топ-5 причин?
Ответ: Вот топ-5 причин, что понравилось посетителям парка:

1. Разнообразие локаций для фото.
2. Чистые туалеты.
3. Вкусное мороженое в японском саду.
4. Детские площадки и фонтаны.
5. Интересные скульптуры и растения.
Вопрос: что не понравилось больше всего, дай топ-5 причин?
Ответ: Я не знаю. В контексте не указаны причины, которые не понравились посетителям парка. Все отзывы положительные и не содержат информации о негативных моментах.
Вопрос: Галицкий, кто это?
Ответ: Я не знаю.


In [43]:
# try to apply RAG-fusion so our LLM could understand everything better.
template = '''You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):'''
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

chain = (
    {'context': pinecone.as_retriever(), 'question': RunnablePassthrough()}
    | prompt_rag_fusion
    | model
    | parser
    | (lambda x: x.split("\n"))
)

In [44]:
# follow rag fusion instructions from langchain documentation:
def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retriever = pinecone.as_retriever()
retrieval_chain_rag_fusion = chain | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

/Users/imac/llm/.venv/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


12

In [45]:
# let us create common chain with rag fusion as well
template = """Отвечай на вопрос в соответствии с заданным контекстом, используй только русский язык:

{context}

Вопрос: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | model
    | parser
)

final_rag_chain.invoke({"question": 'кто такой Галицкий? Что о нём говорят?'})

'В контексте текста Галицкий - это человек, который потратил деньги на создание парка, и, судя по отзыву, эти деньги не были потрачены зря.'

In [46]:
questions = [
    'есть ли в парке мороженое?',
    'можно ли покататься на самокате?',
    'Есть ли минусы у этого парка?',
    'что понравилось больше всего, дай топ-5 причин?',
    'что не понравилось больше всего, дай топ-5 причин?',
    'Галицкий, кто это?'
]

for question in questions:
    print(f'Вопрос: {question}')
    print(f"Ответ: {final_rag_chain.invoke({'question': question})}")

Вопрос: есть ли в парке мороженое?
Ответ: Да, в парке есть мороженое.
Вопрос: можно ли покататься на самокате?
Ответ: В одном из отзывов есть упоминание о том, что в парке нет самокатов для аренды, и автор отзыва предлагает их установить. Поэтому, к сожалению, на данный момент нельзя покататься на самокате в этом парке.
Вопрос: Есть ли минусы у этого парка?
Ответ: В целом, парк имеет положительные отзывы, но есть несколько минусов, которые упоминаются в отзывах. К ним относятся: 

- трудности с обходом всего парка пешком;
- запрет на вход с электросамокатами и собаками;
- необходимость спрашивать и искать туалеты;
- жалобы на то, что люди не умеют вести себя kulturно и оставляют мусор в парке.

Однако, эти минусы не портят общего положительного впечатления от парка.
Вопрос: что понравилось больше всего, дай топ-5 причин?
Ответ: Вот топ-5 причин, которые понравились больше всего:

1. Красота и обширность парка: многие пользователи отметили, что парк является шикарным, огромным и имеет м

In [ ]:
# R E M A R K A B L E! Our children always grow so fast... BTW this kulturно is strange but ok)
# gonna deploy as a telegram bot and have some fun